# Capstone Project: The Battle of Neighbourhoods (WEEK1)


by Fais Chalo

## 1. Introduction


### 1.1. Problem Description

Visiting a new city which seems to be to big to explore all in once is a problem when it comes to metropolies like London. Through a wide range of different cultures London developent economically, socially and politcially.

One great aspect of the diversity is the wide range of different eating possibilities.

But since it can be quite hard to distinguish between a good and bad restaurant and which one to choose there should be parameters in consideration to easen up the search. In my todays analysis i want to take the case of african food restaurants in London, which i recently heard of are quite popular. 

### 1.2 Background and Motivation

Since my coworker is from Ghana and we recently got a new colleague from Senegal, my interest for the african food quitly increased since there are bringing all kind of delicious food to work. 

My Ghanese coworker mentioned therefore that he is thinking about opening a Ghanese restaurant in London, but isn't quite sure about the current information provided.

Therefore i will lend him a hand in providing a deep analysis about the possibilities.

### 1.3. Possible Audience

Like i mentioned there is a high range of diversity in London and so we can conclude that besides the tourist also locals could be highly interested in finding the most visitable restaurant in whole london. Therefore the target audience are people who care about a high sophisticated african restaurant with high quality standarts.

## 2. Data


### 2.1 Description of Data 

In my Analysis i will take data from the sites Wikipedia and Foursquare.

**Dataset 1**

In this dataset i will examine data from the Areas of London which was provided by wikipedia.

The London Area has 32 Boroughs and the "City of London". You can find the data here - Greater London Area <https://en.wikipedia.org/wiki/List_of_areas_of_London >

The web scrapped of the Wikipedia page for the Greater London Area data is provided below:

In [27]:

import requests
import pandas as pd
from bs4 import BeautifulSoup

In [52]:
wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_areas_of_London'
headers = {'User-Agent': 'Chrome/5.0 (Windows 10.0; Win64; x64; rv:64.0) Fais/ Chrome/64.0'}
wikipedia_page = requests.get(wikipedia_link, headers = headers)
wikipedia_page

<Response [200]>

In [2]:
wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_areas_of_London'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:64.0) Gecko/20100101 Firefox/64.0'}
wikipedia_page = requests.get(wikipedia_link, headers = headers)
wikipedia_page

<Response [200]>

In [31]:
# First you need to clean html file and extract the "tbody" within the table where class is "wikitable sortable"

soup = BeautifulSoup(wikipedia_page.content, 'html.parser')
table = soup.find('table', {'class':'wikitable sortable'}).tbody

In [33]:
# Extracts all "tr" (table rows) within the table above
rows = table.find_all('tr')
# Extracts the column headers, removes and replaces possible '\n' with space for the "th" tag
columns = [i.text.replace('\n', '')
           for i in rows[0].find_all('th')]
# Converts columns to pd dataframe
df = pd.DataFrame(columns = columns)
df

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref


In [34]:
# Extracts every row with corresponding columns, then append the values to the created pd dataframe "df"
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    
    if len(tds) == 7:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', ''.replace('\xa0','')), tds[3].text, tds[4].text.replace('\n', ''.replace('\xa0','')), tds[5].text.replace('\n', ''.replace('\xa0','')), tds[6].text.replace('\n', ''.replace('\xa0',''))]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        
        df = df.append(pd.Series(values, index = columns), ignore_index = True)

        df

In [39]:
df.head(5)

,Location,Borough,Post-town,Postcode,Dial-code,OSGridRef
0,Abbey Wood,"Bexley, Greenwich [1]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[2]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[2],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[2],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [38]:
df = df.rename(index=str, columns = {'Location': 'Location', 'London\xa0borough': 'Borough', 'Post town': 'Post-town', 'Postcode\xa0district': 'Postcode', 'Dial\xa0code': 'Dial-code', 'OS grid ref': 'OSGridRef'})

In [41]:
df['Borough'] = df['Borough'].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))
df0 = df.drop('Postcode', axis=1).join(df['Postcode'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('Postcode'))
df1 = df0[['Location', 'Borough', 'Postcode', 'Post-town']].reset_index(drop=True)

In [42]:
df1.head(5)

,Location,Borough,Postcode,Post-town
0,Abbey Wood,"Bexley, Greenwich",SE2,LONDON
1,Acton,"Ealing, Hammersmith and Fulham",W3,LONDON
2,Acton,"Ealing, Hammersmith and Fulham",W4,LONDON
3,Angel,Islington,EC1,LONDON
4,Angel,Islington,N1,LONDON


In [43]:
df2 = df1
df21 = df2[df2['Post-town'].str.contains('LONDON')]

In [44]:
df21.head(5)

,Location,Borough,Postcode,Post-town
0,Abbey Wood,"Bexley, Greenwich",SE2,LONDON
1,Acton,"Ealing, Hammersmith and Fulham",W3,LONDON
2,Acton,"Ealing, Hammersmith and Fulham",W4,LONDON
3,Angel,Islington,EC1,LONDON
4,Angel,Islington,N1,LONDON


In [45]:
# We need to take of the column "Post-town" out of dataset "df21"
df3 = df21[['Location', 'Borough', 'Postcode']].reset_index(drop=True)

In [46]:
df3.head(10)

,Location,Borough,Postcode
0,Abbey Wood,"Bexley, Greenwich",SE2
1,Acton,"Ealing, Hammersmith and Fulham",W3
2,Acton,"Ealing, Hammersmith and Fulham",W4
3,Angel,Islington,EC1
4,Angel,Islington,N1
5,Church End,Brent,NW10
6,Church End,Barnet,N3
7,Clapham,"Lambeth, Wandsworth",SW4
8,Clerkenwell,Islington,EC1
9,Colindale,Barnet,NW9


In [48]:
df_london = df3

From here you should install geocoder through pip

In [49]:
!pip -q install geocoder
import geocoder

In [50]:
'''Geocoder starts here'''
'''Defining a function to use --> get_latlng()'''
def get_latlng(arcgis_geocoder):
    
    # Initialize the Location (lat. and long.) to "None"
    lat_lng_coords = None
    
    # While loop helps to create a continous run until all the location coordinates are geocoded
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, London, United Kingdom'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords
'''Geocoder ends here'''

'Geocoder ends here'

In [51]:
postal_codes = df_london['Postcode']    

In [24]:
coordinates = [get_latlng(postal_code) for postal_code in postal_codes.tolist()]

In [25]:
# now we store the dataframe into a new line
df_london_loc = df_london

# The obtained coordinates (latitude and longitude) are joined with the dataframe as shown
df_london_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
df_london_loc['Latitude'] = df_london_coordinates['Latitude']
df_london_loc['Longitude'] = df_london_coordinates['Longitude']

In [26]:
df_london_loc.head()

,Location,Borough,Postcode,Latitude,Longitude
0,Abbey Wood,"Bexley, Greenwich",SE2,51.49245,0.12127
1,Acton,"Ealing, Hammersmith and Fulham",W3,51.51324,-0.26746
2,Acton,"Ealing, Hammersmith and Fulham",W4,51.48944,-0.26194
3,Angel,Islington,EC1,51.52361,-0.09877
4,Angel,Islington,N1,51.53791,-0.09984


The data output df_london_loc will be used in week 2.

### 2.1.2 Dataset 2

We will use the Foursquare API to locate the places of neighbourhoods of London.

The places will provide the categories needed for the analysis. Checking through the categories we can answer the question which restaurant is in favor of another one.

## 2.2 How data will be used to solve the problem¶

We will check the datasets 1 and 2 and considered in a certain radius. To be mentioned we will only be able to show off 100 places due to Foursquare resreiction. In the further analysis we will consider proximity/ cost types etc. 